In [1]:
import os
import glob
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from darts import TimeSeries
from sklearn.metrics import mean_absolute_error, mean_squared_error
from torch.utils.data import DataLoader, TensorDataset
import torch
import csv
from darts.dataprocessing.transformers import Scaler
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import matplotlib.pyplot as plt
import torch 
import torch.optim as optim



In [2]:

def compute_metrics(y_true, y_pred):
    """
    Compute MAE, MSE, and RMSE between true and predicted values.
    
    Parameters:
    - y_true (np.ndarray): True values
    - y_pred (np.ndarray): Predicted values
    
    Returns:
    - dict: {'MAE': ..., 'MSE': ..., 'RMSE': ...}
    """
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return {"MAE": mae, "MSE": mse, "RMSE": rmse}


def load_building_series(folder_path):
    all_files = glob.glob(os.path.join(folder_path, "*.csv"))
    series_list = []

    for file in all_files:
        df = pd.read_csv(file, parse_dates=['timestamp'])
        df = df.sort_values('timestamp')
        ts = TimeSeries.from_dataframe(df, 'timestamp', 'kWh')
        series_list.append(ts)

    return series_list


def split_series_list(series_list, train_ratio=0.75):
    train_series = []
    test_series = []
    for ts in series_list:
        train, test = ts.split_before(train_ratio)
        train_series.append(train)
        test_series.append(test)
    return train_series, test_series


from typing import List, Tuple, Optional

def convert_timeseries_to_numpy(
    series: TimeSeries,
    input_len: int,
    output_len: int,
    drop_incomplete: bool = True
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Converts a single Darts TimeSeries into input-output pairs for supervised learning.

    Args:
        series (TimeSeries): The input time series.
        input_len (int): Length of input window.
        output_len (int): Length of output window.
        drop_incomplete (bool): If True, drops windows that can't fit input+output.

    Returns:
        Tuple[np.ndarray, np.ndarray]: Arrays (X, y) where:
            - X shape: (num_samples, input_len, num_features)
            - y shape: (num_samples, output_len, num_features)
    """
    values = series.values()  # shape: (T, D)
    T = len(values)
    max_i = T - input_len - output_len + 1

    if drop_incomplete and max_i <= 0:
        return np.empty((0, input_len, values.shape[1])), np.empty((0, output_len, values.shape[1]))

    X_all, y_all = [], []

    for i in range(max_i):
        x_i = values[i : i + input_len]
        y_i = values[i + input_len : i + input_len + output_len]
        X_all.append(x_i)
        y_all.append(y_i)

    return np.array(X_all), np.array(y_all)


def create_dataloader(X, y, batch_size=32):
    dataset = TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32))
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [3]:
# import pandas as pd

# df = pd.read_csv("train.csv", usecols=["building_id", "timestamp", "meter", "meter_reading"])
# df[df['meter'] == 0].to_feather("meter_0_data.feather")


In [4]:
def load_energy_data_feather(cid, filepath="meter_0_data.feather"):
    """Load, preprocess, and return train/test dataloaders for a client."""
    df = pd.read_feather(filepath)
    df = df[df['building_id'] == cid]
    df['meter_reading'] = df['meter_reading'].fillna(0)

    if df.empty:
        raise ValueError(f"No data found for building_id {cid}")

    try:
        ts = TimeSeries.from_dataframe(
            df,
            time_col='timestamp',
            value_cols='meter_reading',
            fill_missing_dates=True,
            freq='h'
        )
    except Exception as e:
        raise ValueError(f"Failed to construct TimeSeries: {e}")

    train_series, test_series = ts.split_before(0.75)

    if len(train_series) == 0 or len(test_series) == 0:
        raise ValueError(f"Empty time series for building_id {cid}. Train: {len(train_series)}, Test: {len(test_series)}")

    scaler = MinMaxScaler(feature_range=(0.1, 1))
    transformer = Scaler(scaler)
    transformed_train_series = transformer.fit_transform(train_series)
    transformed_test_series = transformer.transform(test_series)

    X_train, y_train = convert_timeseries_to_numpy(transformed_train_series, input_len=24, output_len=8)
    X_test, y_test = convert_timeseries_to_numpy(transformed_test_series, input_len=24, output_len=8)

    X_train = np.nan_to_num(X_train, nan=0.0)
    y_train = np.nan_to_num(y_train, nan=0.0)
    X_test = np.nan_to_num(X_test, nan=0.0)
    y_test = np.nan_to_num(y_test, nan=0.0)


    if len(X_train) == 0 or len(X_test) == 0:
        raise ValueError(f"Client {cid} has no data after preprocessing.")

    train_loader = create_dataloader(X_train, y_train, batch_size=512)
    test_loader = create_dataloader(X_test, y_test, batch_size=256)

    return train_loader, test_loader


In [14]:
filepath="meter_0_data.feather"
cid = 50
# Load and filter by building
df = pd.read_feather(filepath)
# df = df[df['building_id'] == cid]

In [16]:
df.building_id.value_counts

<bound method IndexOpsMixin.value_counts of 0              0
1              1
2              2
3              3
4              4
            ... 
20216095    1444
20216096    1445
20216097    1446
20216098    1447
20216099    1448
Name: building_id, Length: 12060910, dtype: int64>

In [7]:
train_dl, test_dl = load_energy_data_feather(0, filepath="meter_0_data.feather")

In [8]:
from tqdm import tqdm

In [9]:
def train(model, train_loader, device = None, learning_rate=0.001, loss_fn=None, optimizer_class=optim.Adam, epochs=50):
      
        if device == None: 
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        loss_fn = loss_fn if loss_fn is not None else nn.MSELoss()
        optimizer = optimizer_class(model.parameters(), lr=learning_rate)
        model.train()
        avg_loss = 0
        for epoch in tqdm(range(epochs)):
            epoch_loss = 0.0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_batch = y_batch.squeeze(-1) if y_batch.dim() == 3 and y_batch.shape[-1] == 1 else y_batch

                optimizer.zero_grad()
                output = model(X_batch)
                loss = loss_fn(output, y_batch)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            avg_loss = epoch_loss / len(train_loader)

        return avg_loss
            # print(f"[Epoch {epoch + 1}] Training Loss: {avg_loss:.4f}")


In [10]:
from Models import MoELSTM

c:\Users\srina\OneDrive\Documents\Federated Learning - Priyanka\Diffusion_FL\energy-ts-diffusion\darts\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
net = MoELSTM(
            input_size=1, hidden_size=64, output_size=8,
            num_experts=5, ffn_hidden_size=32
        )

In [17]:
exo = []
for i in range(700,1448):
    try:
        train_dl, test_dl = load_energy_data_feather(i, filepath="meter_0_data.feather")
    except:
        print(f"Error in file {i}")
        exo.append(i)
        
    # train(net, train_dl, device = None, learning_rate=0.001, loss_fn=None, optimizer_class=optim.Adam, epochs=5)

    

Error in file 751
Error in file 752
Error in file 754
Error in file 757
Error in file 763
Error in file 772
Error in file 783
Error in file 786
Error in file 789
Error in file 790
Error in file 792
Error in file 933
Error in file 934
Error in file 1072
Error in file 1077
Error in file 1078
Error in file 1085
Error in file 1112
Error in file 1116
Error in file 1132
Error in file 1145
Error in file 1155
Error in file 1180
Error in file 1187
Error in file 1192
Error in file 1204
Error in file 1330
Error in file 1349
Error in file 1354
Error in file 1372
Error in file 1373
Error in file 1374
Error in file 1385
Error in file 1388
Error in file 1397
Error in file 1426


In [18]:
import pandas as pd

# Load the data
df = pd.read_feather("meter_0_data.feather")

# List of building IDs to exclude (the ones without data)
excluded_building_ids = exo # [12, 25, 37, 51]  # Example values, replace with yours

# Filter out the excluded buildings
df_filtered = df[~df['building_id'].isin(excluded_building_ids)].copy()

# Map old building IDs to new consecutive IDs (starting from 0)
unique_ids = sorted(df_filtered['building_id'].unique())
new_id_map = {old_id: new_id for new_id, old_id in enumerate(unique_ids)}
df_filtered['building_id'] = df_filtered['building_id'].map(new_id_map)

# Save the cleaned and updated data back to feather
df_filtered.reset_index(drop=True, inplace=True)
df_filtered.to_feather("meter_0_data_cleaned.feather")

print("Cleaned dataset saved as meter_0_data_cleaned.feather")
print(f"Removed {len(excluded_building_ids)} buildings, now have {len(unique_ids)} total buildings.")


Cleaned dataset saved as meter_0_data_cleaned.feather
Removed 36 buildings, now have 1413 total buildings.


In [13]:
train(net, train_dl, device = None, learning_rate=0.001, loss_fn=None, optimizer_class=optim.Adam, epochs=5)

100%|██████████| 5/5 [00:02<00:00,  1.75it/s]


0.009053892312714687

In [74]:
df['building_id'].unique()

array([  0,   1,   2, ..., 591, 783, 403])

In [ ]:
for i in df['building_id'].unique

In [ ]:
def load_energy_data(cid):

    df = pd.read_csv("train.csv")

    df_ele = df[df.meter == 0]

    temp_df = df_ele[df_ele['building_id']==cid]
    # df = pd.read_csv(f"Dataset/buildings/building_{cid}.csv")

    ts = TimeSeries.from_dataframe(temp_df, 'timestamp', 'meter_reading')
    train_series, test_series = ts.split_before(0.75) 

    scaler = MinMaxScaler(feature_range=(0.1, 1))
    transformer = Scaler(scaler)
    transformed_train_series = transformer.fit_transform(train_series)
    transformed_test_series = transformer.fit_transform(test_series)


    X_train, y_train = convert_timeseries_to_numpy(transformed_train_series, input_len=24, output_len=8)
    X_test, y_test = convert_timeseries_to_numpy(transformed_test_series, input_len=24, output_len=8)

    train_loader = create_dataloader(X_train, y_train, batch_size=512)
    test_loader = create_dataloader(X_test, y_test, batch_size=256)

   
    return train_loader, test_loader


In [49]:
train_dl, test_dl = load_energy_data(1)

In [ ]:
def norm_values():
    

In [13]:
df.meter.value_counts

<bound method IndexOpsMixin.value_counts of 0           0
1           0
2           0
3           0
4           0
           ..
20216095    0
20216096    0
20216097    0
20216098    0
20216099    0
Name: meter, Length: 20216100, dtype: int64>

In [1]:
def make_is_bad_zero(Xy_subset, min_interval=48, summer_start=3000, summer_end=7500):
    """Helper routine for 'find_bad_zeros'.
    
    This operates upon a single dataframe produced by 'groupby'. We expect an 
    additional column 'meter_id' which is a duplicate of 'meter' because groupby 
    eliminates the original one."""
    meter = Xy_subset.meter_id.iloc[0]
    is_zero = Xy_subset.meter_reading == 0
    if meter == 0:
        # Electrical meters should never be zero. Keep all zero-readings in this table so that
        # they will all be dropped in the train set.
        return is_zero

    transitions = (is_zero != is_zero.shift(1))
    all_sequence_ids = transitions.cumsum()
    ids = all_sequence_ids[is_zero].rename("ids")
    if meter in [2, 3]:
        # It's normal for steam and hotwater to be turned off during the summer
        keep = set(ids[(Xy_subset.timestamp < summer_start) |
                       (Xy_subset.timestamp > summer_end)].unique())
        is_bad = ids.isin(keep) & (ids.map(ids.value_counts()) >= min_interval)
    elif meter == 1:
        time_ids = ids.to_frame().join(Xy_subset.timestamp).set_index("timestamp").ids
        is_bad = ids.map(ids.value_counts()) >= min_interval

        # Cold water may be turned off during the winter
        jan_id = time_ids.get(0, False)
        dec_id = time_ids.get(8283, False)
        if (jan_id and dec_id and jan_id == time_ids.get(500, False) and
                dec_id == time_ids.get(8783, False)):
            is_bad = is_bad & (~(ids.isin(set([jan_id, dec_id]))))
    else:
        raise Exception(f"Unexpected meter type: {meter}")

    result = is_zero.copy()
    result.update(is_bad)
    return result

def find_bad_zeros(X, y):
    """Returns an Index object containing only the rows which should be deleted."""
    Xy = X.assign(meter_reading=y, meter_id=X.meter)
    is_bad_zero = Xy.groupby(["building_id", "meter"]).apply(make_is_bad_zero)
    return is_bad_zero[is_bad_zero].index.droplevel([0, 1])

`find_bad_sitezero` identifies the "known-bad" electrical readings from the first 141 days of the data for site 0 (i.e. UCF).

In [2]:
def find_bad_sitezero(X):
    """Returns indices of bad rows from the early days of Site 0 (UCF)."""
    return X[(X.timestamp < 3378) & (X.site_id == 0) & (X.meter == 0)].index

`find_bad_building1099` identifies the most absurdly high readings from building 1099. These are orders of magnitude higher than all data, and have been emperically seen in LB probes to be harmful outliers.

In [3]:
def find_bad_building1099(X, y):
    """Returns indices of bad rows (with absurdly high readings) from building 1099."""
    return X[(X.building_id == 1099) & (X.meter == 2) & (y > 3e4)].index

Finally, `find_bad_rows` combines all of the above together to allow you to do a one-line cleanup of your data.

In [4]:
def find_bad_rows(X, y):
    return find_bad_zeros(X, y).union(find_bad_sitezero(X)).union(find_bad_building1099(X, y))

# Framework
The following code is taken from my previous kernel: [Strategy evaluation: What helps and by how much?](https://www.kaggle.com/purist1024/strategy-evaluation-what-helps-and-by-how-much). It is described in more detail there and so, in order to get to the point, we incorporate it here without the descriptions.

In [5]:
import pandas as pd
import numpy as np
import os
import warnings

from lightgbm import LGBMRegressor
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.metrics import mean_squared_log_error

pd.set_option("max_columns", 500)

def input_file(file):
    path = f"../input/ashrae-energy-prediction/{file}"
    if not os.path.exists(path): return path + ".gz"
    return path

def compress_dataframe(df):
    result = df.copy()
    for col in result.columns:
        col_data = result[col]
        dn = col_data.dtype.name
        if dn == "object":
            result[col] = pd.to_numeric(col_data.astype("category").cat.codes, downcast="integer")
        elif dn == "bool":
            result[col] = col_data.astype("int8")
        elif dn.startswith("int") or (col_data.round() == col_data).all():
            result[col] = pd.to_numeric(col_data, downcast="integer")
        else:
            result[col] = pd.to_numeric(col_data, downcast='float')
    return result

def read_train():
    df = pd.read_csv(input_file("train.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    return compress_dataframe(df)

def read_building_metadata():
    return compress_dataframe(pd.read_csv(
        input_file("building_metadata.csv")).fillna(-1)).set_index("building_id")

site_GMT_offsets = [-5, 0, -7, -5, -8, 0, -5, -5, -5, -6, -7, -5, 0, -6, -5, -5]

def read_weather_train(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    df = pd.read_csv(input_file("weather_train.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    if fix_timestamps:
        GMT_offset_map = {site: offset for site, offset in enumerate(site_GMT_offsets)}
        df.timestamp = df.timestamp + df.site_id.map(GMT_offset_map)
    if interpolate_na:
        site_dfs = []
        for site_id in df.site_id.unique():
            # Make sure that we include all possible hours so that we can interpolate evenly
            site_df = df[df.site_id == site_id].set_index("timestamp").reindex(range(8784))
            site_df.site_id = site_id
            for col in [c for c in site_df.columns if c != "site_id"]:
                if add_na_indicators: site_df[f"had_{col}"] = ~site_df[col].isna()
                site_df[col] = site_df[col].interpolate(limit_direction='both', method='linear')
                # Some sites are completely missing some columns, so use this fallback
                site_df[col] = site_df[col].fillna(df[col].median())
            site_dfs.append(site_df)
        df = pd.concat(site_dfs).reset_index()  # make timestamp back into a regular column
    elif add_na_indicators:
        for col in df.columns:
            if df[col].isna().any(): df[f"had_{col}"] = ~df[col].isna()
    return compress_dataframe(df).set_index(["site_id", "timestamp"])

def combined_train_data(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    Xy = compress_dataframe(read_train().join(read_building_metadata(), on="building_id").join(
        read_weather_train(fix_timestamps, interpolate_na, add_na_indicators),
        on=["site_id", "timestamp"]).fillna(-1))
    return Xy.drop(columns=["meter_reading"]), Xy.meter_reading

def _add_time_features(X):
    return X.assign(tm_day_of_week=((X.timestamp // 24) % 7), tm_hour_of_day=(X.timestamp % 24))

class CatSplitRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, model, col):
        self.model = model
        self.col = col

    def fit(self, X, y):
        self.fitted = {}
        importances = []
        for val in X[self.col].unique():
            X1 = X[X[self.col] == val].drop(columns=[self.col])
            self.fitted[val] = clone(self.model).fit(X1, y.reindex_like(X1))
            importances.append(self.fitted[val].feature_importances_)
            del X1
        fi = np.average(importances, axis=0)
        col_index = list(X.columns).index(self.col)
        self.feature_importances_ = [*fi[:col_index], 0, *fi[col_index:]]
        return self

    def predict(self, X):
        result = np.zeros(len(X))
        for val in X[self.col].unique():
            ix = np.nonzero((X[self.col] == val).to_numpy())
            predictions = self.fitted[val].predict(X.iloc[ix].drop(columns=[self.col]))
            result[ix] = predictions
        return result

categorical_columns = [
    "building_id", "meter", "site_id", "primary_use", "had_air_temperature", "had_cloud_coverage",
    "had_dew_temperature", "had_precip_depth_1_hr", "had_sea_level_pressure", "had_wind_direction",
    "had_wind_speed", "tm_day_of_week", "tm_hour_of_day"
]

class LGBMWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, categorical_feature=None, **params):
        self.model = LGBMRegressor(**params)
        self.categorical_feature = categorical_feature

    def fit(self, X, y):
        with warnings.catch_warnings():
            cats = None if self.categorical_feature is None else list(
                X.columns.intersection(self.categorical_feature))
            warnings.filterwarnings("ignore",
                                    "categorical_feature in Dataset is overridden".lower())
            self.model.fit(X, y, **({} if cats is None else {"categorical_feature": cats}))
            self.feature_importances_ = self.model.feature_importances_
            return self

    def predict(self, X):
        return self.model.predict(X)

    def get_params(self, deep=True):
        return {**self.model.get_params(deep), "categorical_feature": self.categorical_feature}

    def set_params(self, **params):
        ctf = params.pop("categorical_feature", None)
        if ctf is not None: self.categorical_feature = ctf
        self.model.set_params(params)

The following functions are simple variants of the ones above, but deal with loading in the test set. They could easily be refactored to share code with those functions, but we keep them separate for this demonstration.

In [6]:
def read_test():
    df = pd.read_csv(input_file("test.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    return compress_dataframe(df).set_index("row_id")

def read_weather_test(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    df = pd.read_csv(input_file("weather_test.csv"), parse_dates=["timestamp"])
    df.timestamp = (df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
    if fix_timestamps:
        GMT_offset_map = {site: offset for site, offset in enumerate(site_GMT_offsets)}
        df.timestamp = df.timestamp + df.site_id.map(GMT_offset_map)
    if interpolate_na:
        site_dfs = []
        for site_id in df.site_id.unique():
            # Make sure that we include all possible hours so that we can interpolate evenly
            site_df = df[df.site_id == site_id].set_index("timestamp").reindex(range(8784, 26304))
            site_df.site_id = site_id
            for col in [c for c in site_df.columns if c != "site_id"]:
                if add_na_indicators: site_df[f"had_{col}"] = ~site_df[col].isna()
                site_df[col] = site_df[col].interpolate(limit_direction='both', method='linear')
                # Some sites are completely missing some columns, so use this fallback
                site_df[col] = site_df[col].fillna(df[col].median())
            site_dfs.append(site_df)
        df = pd.concat(site_dfs).reset_index()  # make timestamp back into a regular column
    elif add_na_indicators:
        for col in df.columns:
            if df[col].isna().any(): df[f"had_{col}"] = ~df[col].isna()
    return compress_dataframe(df).set_index(["site_id", "timestamp"])

def combined_test_data(fix_timestamps=True, interpolate_na=True, add_na_indicators=True):
    X = compress_dataframe(read_test().join(read_building_metadata(), on="building_id").join(
        read_weather_test(fix_timestamps, interpolate_na, add_na_indicators),
        on=["site_id", "timestamp"]).fillna(-1))
    return X

# Fit, predict, and submit


First, read in the data, and identify the bad rows using the provided functions. Write out the bad rows in an index-per-line format for fast and easy re-use.

In [7]:
X, y = combined_train_data()

bad_rows = find_bad_rows(X, y)
pd.Series(bad_rows.sort_values()).to_csv("rows_to_drop.csv", header=False, index=False)

Drop the bad rows that we identified above, and then train the model using our favorite features and regressors. See [Strategy evaluation: What helps and by how much?](https://www.kaggle.com/purist1024/strategy-evaluation-what-helps-and-by-how-much) for information on the specific strategies.

In [8]:
X = X.drop(index=bad_rows)
y = y.reindex_like(X)

# Additional preprocessing
X = compress_dataframe(_add_time_features(X))
X = X.drop(columns="timestamp")  # Raw timestamp doesn't help when prediction
y = np.log1p(y)

model = CatSplitRegressor(
    LGBMWrapper(random_state=0, n_jobs=-1, categorical_feature=categorical_columns), "meter")

model.fit(X, y)
del X, y

Load the test set and predict meter readings. We must, of course, use exponentiation to convert our predictions back from log-scale to the desired kWh values. We also clip to a minimum of zero, since we know that there will be no negative readings.

In [9]:
X = combined_test_data()
X = compress_dataframe(_add_time_features(X))
X = X.drop(columns="timestamp")  # Raw timestamp doesn't help when prediction

predictions = pd.DataFrame({
    "row_id": X.index,
    "meter_reading": np.clip(np.expm1(model.predict(X)), 0, None)
})

del X

Finally, write the predictions out for submission. After that, it's Miller Time (tm).

In [10]:
predictions.to_csv("submission.csv", index=False, float_format="%.4f")